<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Introduction and basic usage

#### Imports and external functions

In [ ]:
import io
from pathlib import Path
from IPython.display import Image, display
from pprint import pprint
import numpy as np

In [ ]:
def plot_state_machine(pandora2d_machine):
    """
    Show the schemes of step of Pandora2D Machine
    """
    stream = io.BytesIO()
    try:
        pandora2d_machine.get_graph().draw(stream, prog='dot', format='png')
        display(Image(stream.getvalue()))
    except:
        print("It is not possible to show the graphic of the state machine. To solve it, please install graphviz on your system (apt-get install graphviz if operating in Linux) and install python package with pip insall graphviz")

In [ ]:
from snippets.utils import *

# What is Pandora2D ?
* Pandora2d is a Toolbox to estimate disparity in two dimensions
* It works with Pandora
* It is easy to configure and modular

## Inputs

* Image pair
* Value associated to no_data images
* Disparity ranges to explore (if not estimation step)
* Configuration file

## Outputs

* Disparity maps for row's axis
* Disparity maps for column's axis

## Pandora2D's pipeline

Pandora2D provides the following steps:
* estimation computation
* matching cost computation 
* disparity computation (**mandatory if matching_cost**)
* subpixel disparity refinement

<img src="img/Pandora2D_pipeline.drawio.svg" width="1000">

### Available implementations for each step

| Step                          | Algorithms implemented                   |
|:------------------------------|:-----------------------------------------|
| Estimation computation        | phase cross correlation                  |
| Matching cost computation     | SAD / SSD / ZNNC                         |
| Disparity computation         | Winner-Takes-All                         |
| Subpixel disparity refinement | Dichotomy / Optical flow                 |

# Pandora2D execution options with state machine

#### Imports of pandora2d

In [ ]:
# Load pandora2d imports
import pandora2d
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import create_datasets_from_inputs

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

Provide output directory to write results

In [ ]:
output_dir = Path.cwd() / "output"
# If necessary, create output dir
output_dir.mkdir(exist_ok=True,parents=True)

Convert input data to dataset

In [ ]:
input_config = {
    "left": {"img": img_left_path, "nodata": np.nan},
    "right": {"img": img_right_path, "nodata": np.nan},
    "col_disparity": {"init": 0, "range": 2},
    "row_disparity": {"init": 0, "range": 2},
}

In [ ]:
img_left, img_right = create_datasets_from_inputs(input_config=input_config)

`create_datasets_from_inputs` returns a namedTuple so we could have used:

```python
image_datasets = create_datasets_from_inputs(input_config=input_config)
```

and called:

 `image_datasets.left` or `image_datasets.right` instead of `img_left` and `img_right`.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(1,2,1)
ax0.imshow(img_left["im"].data)
plt.title("Left image")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(img_right["im"].data)
plt.title("Right image")

# Option 1 : trigger all the steps of the machine at ones

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": "data/left.tif",
            "nodata": "NaN",
        },
        "right": {
            "img": "data/right.tif",
        },
        "col_disparity": {"init": 0, "range": 2},
        "row_disparity": {"init": 0, "range": 2},
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -5
        },
        "refinement" : {
            "refinement_method": "dichotomy",
            "iterations": 2,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/introduction_and_basic_usage_option_1"
    },
}

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_conf(user_cfg, pandora2d_machine)

In [ ]:
pipeline_cfg = checked_cfg['pipeline']
pprint(pipeline_cfg)

#### Prepare the machine 

In [ ]:
image_datasets = create_datasets_from_inputs(input_config=checked_cfg["input"])

#### Trigger all the steps of the machine at ones

In [ ]:
dataset, _ = pandora2d.run(
    pandora2d_machine,
    image_datasets.left,
    image_datasets.right,
    checked_cfg
    )

Visualize output disparity map

In [ ]:
plot_two_images(dataset["row_map"].data,
                dataset["col_map"].data,
                "Row disparity map",
                "Columns disparity map", 
                output_dir, 
                cmap=pandora_cmap())

Visualize correlation score 

In [ ]:
plot_image(dataset["correlation_score"].data, "Correlation score", output_dir, cmap=pandora_cmap())

# Option 2 : trigger the machine step by step

The implementation of Pandora2D with a state machine makes it possible to set up a more flexible pipeline, which makes it possible to choose via a configuration file the steps wishes to follow in Pandora2D.

Moreover, the state machine allows to run each step of the pipeline independently, giving the possibility to save and visualize the results after each step. 

The state machine has three states : 
* Begin
* Assumption
* Cost volumes
* Disparity maps

Being the connections between them the different steps of the pipeline.

<img src="img/Pandora2D_pipeline.drawio.svg" width="1000">

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": "data/left.tif",
            "nodata": "NaN",
        },
        "right": {
            "img": "data/right.tif",
        },
        "col_disparity": {"init": 0, "range": 2},
        "row_disparity": {"init": 0, "range": 2},
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -5
        },
        "refinement":{
            "refinement_method": "dichotomy",
            "iterations": 2,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/introduction_and_basic_usage_option_2"
    },
}

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_conf(user_cfg, pandora2d_machine)

In [ ]:
pipeline_cfg = checked_cfg['pipeline']
pprint(pipeline_cfg)

In [ ]:
image_datasets = create_datasets_from_inputs(input_config=checked_cfg["input"])

#### Prepare the machine 

In [ ]:
pandora2d_machine.run_prepare(image_datasets.left, image_datasets.right, checked_cfg)

#### Trigger the machine step by step

In [ ]:
plot_state_machine(pandora2d_machine)

Run matching cost 

In [ ]:
pandora2d_machine.run('matching_cost', checked_cfg)

In [ ]:
plot_state_machine(pandora2d_machine)

Run disparity 

In [ ]:
pandora2d_machine.run('disparity', checked_cfg)

In [ ]:
plot_state_machine(pandora2d_machine)

In [ ]:
plot_two_images(pandora2d_machine.dataset_disp_maps["row_map"].data,
                pandora2d_machine.dataset_disp_maps["col_map"].data,
                "Row disparity map",
                "Columns disparity map", 
                output_dir, 
                cmap=pandora_cmap())

Run refinement 

In [ ]:
pandora2d_machine.run('refinement', checked_cfg)

In [ ]:
plot_state_machine(pandora2d_machine)

Visualize output disparity map

In [ ]:
plot_two_images(pandora2d_machine.dataset_disp_maps["row_map"].data,
                pandora2d_machine.dataset_disp_maps["col_map"].data,
                "Row refined disparity map",
                "Columns refined disparity map", 
                output_dir, 
                cmap=pandora_cmap())